## Pre-processing

In [9]:
import pandas as pd

df=pd.read_csv('covid.csv') #reading the CDC dataset 

In [10]:
df[['Data as of', 'Start Week', 'End Week', 'State', 'Condition Group',
       'Condition', 'Age Group', 'COVID-19 Deaths']].sample(5)

,Data as of,Start Week,End Week,State,Condition Group,Condition,Age Group,COVID-19 Deaths
667,01/10/2021,01/04/2020,01/09/2021,AK,"Intentional and unintentional injury, poisonin...","Intentional and unintentional injury, poisonin...",85+,0.0
1817,01/10/2021,01/04/2020,01/09/2021,CT,"Intentional and unintentional injury, poisonin...","Intentional and unintentional injury, poisonin...",85+,33.0
943,01/10/2021,01/04/2020,01/09/2021,AR,Respiratory diseases,Adult respiratory distress syndrome,45-54,35.0
2148,01/10/2021,01/04/2020,01/09/2021,DC,Circulatory diseases,Ischemic heart disease,Not stated,0.0
725,01/10/2021,01/04/2020,01/09/2021,AZ,Respiratory diseases,Respiratory failure,65-74,810.0


In [11]:
df1=pd.read_csv('dataset.csv') #reading the public health dataset

In [12]:
df1.columns

Index(['FIPS Code', 'County Name', 'State Abbreviation', 'State Name',
       'CV Death per 100K', 'Population', 'Poverty %', '% Older than 65',
       'Median Household Income', 'Median Home Value',
       'Food Stamp Recipient %', 'Edu. < College %', 'Edu. < Highschool %',
       '# of Health Centers', 'People/Health Centers'],
      dtype='object')

In [13]:
col=df1.columns

In [14]:
col[4:-1]

Index(['CV Death per 100K', 'Population', 'Poverty %', '% Older than 65',
       'Median Household Income', 'Median Home Value',
       'Food Stamp Recipient %', 'Edu. < College %', 'Edu. < Highschool %',
       '# of Health Centers'],
      dtype='object')

## Filling the missing values in the Public health dataset

All numerical feature NaNs are replaced with the means of the corresponding columns.

In [15]:

for i in col[4:-1]:
    mean = df1[i].mean() #filling missing values with mean of corresponding columns
    df1[i].fillna(mean, inplace=True)

In [16]:
df1['People/Health Centers']=df1['Population']/df1['# of Health Centers']

In [17]:
df1.isnull().sum()

FIPS Code                  0
County Name                2
State Abbreviation         2
State Name                 2
CV Death per 100K          0
Population                 0
Poverty %                  0
% Older than 65            0
Median Household Income    0
Median Home Value          0
Food Stamp Recipient %     0
Edu. < College %           0
Edu. < Highschool %        0
# of Health Centers        0
People/Health Centers      0
dtype: int64

In [45]:
df1=df1.dropna() #dropping the two null instances
df1

,FIPS Code,County Name,State Abbreviation,State Name,CV Death per 100K,Population,Poverty %,% Older than 65,Median Household Income,Median Home Value,Food Stamp Recipient %,Edu. < College %,Edu. < Highschool %,# of Health Centers,People/Health Centers
0,2013,Aleutians East,AK,Alaska,106.1,3338.0,14.8,8.2,64000.0,124700.0,3.8,88.5,14.2,6.000000,556.333333
1,8097,Pitkin,CO,Colorado,133.8,17747.0,6.9,16.6,74600.0,593600.0,1.2,38.8,4.7,1.000000,17747.000000
2,8037,Eagle,CO,Colorado,157.5,53726.0,7.9,9.4,79600.0,471100.0,2.7,54.9,10.1,5.000000,10745.200000
3,8117,Summit,CO,Colorado,171.8,29722.0,7.9,11.3,70300.0,547700.0,3.0,52.2,6.6,8.000000,3715.250000
4,2016,Aleutians West,AK,Alaska,184.7,5784.0,8.0,5.5,78200.0,238800.0,2.4,83.8,11.7,6.000000,964.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,22021,Caldwell,LA,Louisiana,876.6,10000.0,22.6,17.2,35400.0,67800.0,22.1,87.6,23.1,6.332356,1579.191108
3139,29221,Washington,MO,Missouri,878.7,24968.0,22.0,15.6,35700.0,90400.0,25.9,91.1,23.0,4.000000,6242.000000
3140,28083,Leflore,MS,Mississippi,880.5,30113.0,35.6,13.1,27600.0,76100.0,38.7,82.1,21.8,1.000000,30113.000000
3141,22025,Catahoula,LA,Louisiana,881.7,10044.0,27.5,15.9,33200.0,78700.0,25.0,86.1,22.6,4.000000,2511.000000


## Featuring engineering 

All the numerical features are calculated for each state by grouping the counties.
Percentages are first converted to actual vales and then grouped based on State codes. These values are then converted to percentages.

In [15]:
#coverting percentages into real values
df1['poverty']=df1['Poverty %']*(df1['Population']/100)
df1['>65']=df1['% Older than 65']*(df1['Population']/100)
df1['Food']=df1['Food Stamp Recipient %']*(df1['Population']/100)
df1['College']=df1['Edu. < College %']*(df1['Population']/100)
df1['highschool']=df1['Edu. < Highschool %']*(df1['Population']/100)
df1['CV deaths']=df1['CV Death per 100K']*(df1['Population']/100000)


In [18]:
#grouping by states 
preprocessed=pd.DataFrame(df1.groupby('State Abbreviation')['Population','poverty', '>65','Food', 'College', 'highschool','CV deaths'].sum())

In [17]:
#converting them back to percentages
preprocessed['poverty %']=(preprocessed['poverty']/preprocessed['Population'])*100
preprocessed['% older than 65 years']=(preprocessed['>65']/preprocessed['Population'])*100
preprocessed['Food Stamp Recipient %']=(preprocessed['Food']/preprocessed['Population'])*100
preprocessed['Edu. < College %']=(preprocessed['College']/preprocessed['Population'])*100
preprocessed['Edu. < Highschool %']=(preprocessed['College']/preprocessed['Population'])*100
preprocessed['Edu. < Highschool %']=(preprocessed['College']/preprocessed['Population'])*100
preprocessed['CV Death per 100K']=(preprocessed['CV deaths']*100000)/preprocessed['Population']

In [18]:
df=df.drop('Flag',axis=1)

In [19]:
df = df[df.State != 'US'] #removing rows that have 'State'='US'


In [20]:
preprocessed.reset_index(inplace=True)


In [21]:
preprocessed = preprocessed.rename(columns={'State Abbreviation': 'State'})


In [22]:
#merging both datasets
final=pd.merge(df, preprocessed, on="State")


In [23]:
final.columns

Index(['Data as of', 'Start Week', 'End Week', 'State', 'Condition Group',
       'Condition', 'ICD10_codes', 'Age Group', 'COVID-19 Deaths',
       'Number of Mentions', 'Population', 'poverty', '>65', 'Food', 'College',
       'highschool', 'CV deaths', 'poverty %', '% older than 65 years',
       'Food Stamp Recipient %', 'Edu. < College %', 'Edu. < Highschool %',
       'CV Death per 100K'],
      dtype='object')

In [24]:
final=final.drop(['poverty', '>65', 'Food', 'College',
       'highschool', 'CV deaths'], axis=1) #removing real value columns

## Final Dataset= Covid, Mask Mandate and Public Health data meged on State codes

In [25]:
final

,Data as of,Start Week,End Week,State,Condition Group,Condition,ICD10_codes,Age Group,COVID-19 Deaths,Number of Mentions,Population,poverty %,% older than 65 years,Food Stamp Recipient %,Edu. < College %,Edu. < Highschool %,CV Death per 100K
0,01/10/2021,01/04/2020,01/09/2021,AL,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,NaN,NaN,4795735.0,17.268944,15.722246,17.824077,75.348869,75.348869,561.024618
1,01/10/2021,01/04/2020,01/09/2021,AL,Respiratory diseases,Influenza and pneumonia,J09-J18,25-34,20.0,21.0,4795735.0,17.268944,15.722246,17.824077,75.348869,75.348869,561.024618
2,01/10/2021,01/04/2020,01/09/2021,AL,Respiratory diseases,Influenza and pneumonia,J09-J18,35-44,34.0,34.0,4795735.0,17.268944,15.722246,17.824077,75.348869,75.348869,561.024618
3,01/10/2021,01/04/2020,01/09/2021,AL,Respiratory diseases,Influenza and pneumonia,J09-J18,45-54,90.0,91.0,4795735.0,17.268944,15.722246,17.824077,75.348869,75.348869,561.024618
4,01/10/2021,01/04/2020,01/09/2021,AL,Respiratory diseases,Influenza and pneumonia,J09-J18,55-64,216.0,224.0,4795735.0,17.268944,15.722246,17.824077,75.348869,75.348869,561.024618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11955,01/10/2021,01/04/2020,01/09/2021,PR,COVID-19,COVID-19,U071,65-74,321.0,321.0,3263022.0,15.883556,18.054035,14.399250,75.324192,75.324192,295.945035
11956,01/10/2021,01/04/2020,01/09/2021,PR,COVID-19,COVID-19,U071,75-84,366.0,366.0,3263022.0,15.883556,18.054035,14.399250,75.324192,75.324192,295.945035
11957,01/10/2021,01/04/2020,01/09/2021,PR,COVID-19,COVID-19,U071,85+,264.0,264.0,3263022.0,15.883556,18.054035,14.399250,75.324192,75.324192,295.945035
11958,01/10/2021,01/04/2020,01/09/2021,PR,COVID-19,COVID-19,U071,Not stated,0.0,0.0,3263022.0,15.883556,18.054035,14.399250,75.324192,75.324192,295.945035


In [26]:
final.to_csv('final_dataset.csv',index=False, header=True)